# LangChain v1.0 가이드

- **LangChain**은 LLM(Large Language Model) 기반 애플리케이션을 쉽게 개발할 수 있도록 도와주는 프레임워크

- 학습 목표
    - LangChain v1.0의 핵심 개념과 아키텍처 이해
    - 에이전트, 모델, 도구, 메시지 등 핵심 컴포넌트 활용
    - 구조화된 출력, 스트리밍, 메모리 관리 등 고급 기능 구현

---

## 1. LangChain 소개

- 핵심 특징:

    1. **간편한 시작**
        - 10줄 미만의 코드로 에이전트 구축 가능
        - OpenAI, Anthropic, Google 등 주요 LLM 제공업체 지원

    2. **유연한 확장**
        - 모듈식 아키텍처로 컴포넌트 교체 용이
        - 다양한 통합(700개 이상) 지원

    3. **프로덕션 준비**
        - LangGraph 기반의 안정적인 실행 환경
        - 스트리밍, 지속성, Human-in-the-loop 지원

- 타임라인:
    - **2022년 10월**: LangChain v0.0.1 출시 (ChatGPT 출시 1개월 전)
    - **2023년 1월**: OpenAI Chat Completion API 출시, 메시지 기반 인터페이스로 전환
    - **2024년 6월**: 700개 이상의 통합 지원, 통합 패키지 분리
    - **2024년 10월**: LangGraph가 복잡한 애플리케이션 구축의 선호 방식으로 자리잡음
    - **2025년 1월**: **LangChain v1.0 출시** - 에이전트 중심의 대대적 개편

- LangChain의 핵심 신념:
    - **LLM은 강력한 새로운 기술**
    - **외부 데이터와 결합하면 더욱 강력해짐**
    - **미래의 애플리케이션은 점점 더 에이전틱(agentic)해질 것**
    - **아직 초기 단계이며, 프로덕션급 에이전트 구축은 여전히 어려움**

- 패키지 재구성

    <div style="background-color: #fff3cd; padding: 15px; border-radius: 5px; margin: 10px 0;">

    ⚠️ **중요 변경사항**

    **`langchain` 패키지** - 핵심 기능만 포함:
    - `langchain.agents`: 에이전트 생성 및 상태 관리
    - `langchain.messages`: 메시지 타입, 콘텐츠 블록
    - `langchain.tools`: 도구 데코레이터, 베이스 클래스
    - `langchain.chat_models`: 통합 모델 초기화
    - `langchain.embeddings`: 임베딩 모델

    **`langchain-classic` 패키지** - 레거시 기능:
    - 레거시 체인 (`LLMChain`, `ConversationChain` 등)
    - 리트리버 (`MultiQueryRetriever` 등)
    - 인덱싱 API
    - Hub 모듈
    - 커뮤니티 통합 재수출

    </div>


---

## 2. 환경 설정

- 기본 패키지 설치

    ```bash
    # pip 사용
    pip install -U langchain
    # Python 3.10 이상 필요

    # uv 사용 (권장)
    uv add langchain
    ```

- 통합 패키지 설치

    ```bash
    # OpenAI 통합
    pip install -U langchain-openai

    # Google 통합
    pip install -U langchain-google-genai
    ```

- API 키 설정

    ```bash
    # .env 파일 생성
    echo "OPENAI_API_KEY=your-api-key-here" > .env
    echo "GOOGLE_API_KEY=your-api-key-here" >> .env
    ```

In [ ]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# API 키 확인
print(f"OpenAI API 키 설정됨: {bool(os.getenv('OPENAI_API_KEY'))}")
print(f"Google API 키 설정됨: {bool(os.getenv('GOOGLE_API_KEY'))}")


In [ ]:
# v1.0의 새로운 에이전트 추상화

from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """주어진 도시의 날씨를 가져옵니다."""
    return f"{city}는 항상 맑습니다!"

agent = create_agent(
    model="gpt-4.1-nano",
    tools=[get_weather],
    system_prompt="당신은 도움이 되는 어시스턴트입니다",
)

# 에이전트 실행
result = agent.invoke(
    {"messages": [{"role": "user", "content": "샌프란시스코의 날씨는 어때?"}]}
)

print(result)

### [연습문제] 환경 설정

- **문제**: 다음 단계를 수행하세요.

    1. `langchain`과 `langchain-google-genai` 패키지를 설치하세요.
    2. `.env` 파일을 생성하고 Google API 키를 설정하세요.
    3. 위의 첫 번째 에이전트 코드를 실행하고 결과를 확인하세요.
    4. `get_weather` 함수를 수정하여 여러 도시에 대해 다른 날씨를 반환하도록 만드세요.


In [ ]:
from dotenv import load_dotenv
from langchain.agents import create_agent

# 환경 변수 로드
load_dotenv()

# 개선된 날씨 함수
def get_weather(city: str) -> str:
    """주어진 도시의 날씨를 가져옵니다."""
    weather_data = {
        "서울": "흐리고 비가 올 예정입니다. 기온은 18도입니다.",
        "부산": "맑고 화창합니다. 기온은 22도입니다.",
        "제주": "구름이 많고 바람이 붑니다. 기온은 20도입니다.",
        "샌프란시스코": "안개가 끼고 선선합니다. 기온은 15도입니다.",
    }
    return weather_data.get(city, f"{city}의 날씨 정보를 찾을 수 없습니다.")

agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    tools=[get_weather],
    system_prompt="당신은 친절한 날씨 정보 제공 어시스턴트입니다.",
)

# 테스트
cities = ["서울", "부산", "샌프란시스코", "도쿄"]
for city in cities:
    result = agent.invoke(
        {"messages": [{"role": "user", "content": f"{city}의 날씨는?"}]}
    )
    print(f"\n{city}: {result['messages'][-1].content}")# 여기에 코드를 작성하세요.


<details>
<summary>✅ 예시 정답</summary>

```python
from dotenv import load_dotenv
from langchain.agents import create_agent

# 환경 변수 로드
load_dotenv()

# 개선된 날씨 함수
def get_weather(city: str) -> str:
    """주어진 도시의 날씨를 가져옵니다."""
    weather_data = {
        "서울": "흐리고 비가 올 예정입니다. 기온은 18도입니다.",
        "부산": "맑고 화창합니다. 기온은 22도입니다.",
        "제주": "구름이 많고 바람이 붑니다. 기온은 20도입니다.",
        "샌프란시스코": "안개가 끼고 선선합니다. 기온은 15도입니다.",
    }
    return weather_data.get(city, f"{city}의 날씨 정보를 찾을 수 없습니다.")

agent = create_agent(
    model="google_genai:gemini-2.5-flash-lite",
    tools=[get_weather],
    system_prompt="당신은 친절한 날씨 정보 제공 어시스턴트입니다.",
)

# 테스트
cities = ["서울", "부산", "샌프란시스코", "도쿄"]
for city in cities:
    result = agent.invoke(
        {"messages": [{"role": "user", "content": f"{city}의 날씨는?"}]}
    )
    print(f"\n{city}: {result['messages'][-1].content}")
```

</details>

---

## 3. 핵심 컴포넌트

### 3.1 모델 (Models)

LangChain은 다양한 LLM과 통신할 수 있는 **통합된 인터페이스**를 제공합니다.


#### 3.1.1 모델 초기화 방법

- **방법 1: 모델 식별자 문자열 사용**

In [ ]:
from langchain.agents import create_agent

# 자동 추론 
agent = create_agent(
    model="gpt-4.1-nano",  # 자동으로 "openai:gpt-4o-nano"로 추론
)

In [ ]:
agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "안녕하세요."
            }
        ]
    }
)

- **방법 2: init_chat_model 사용**

In [ ]:
from langchain.chat_models import init_chat_model

# 명시적 초기화
model = init_chat_model(
    model="gpt-4.1-nano",
    temperature=0.7,
    max_tokens=1000
)

In [ ]:
model.invoke("안녕하세요.")

- **방법 3: 제공자 패키지 직접 사용**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# OpenAI
openai_model = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.7
)

# Google Gemini
gemini_model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7
)

In [ ]:
openai_model.invoke("안녕하세요")

In [ ]:
gemini_model.invoke("안녕하세요")

#### 3.1.2 스트리밍

실시간으로 응답을 받을 수 있습니다:

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano")

# 스트리밍 방식
for chunk in model.stream("앵무새의 깃털이 화려한 이유는 무엇인가요?"):
    print(chunk.text, end="|", flush=True)

#### 3.1.3 도구 호출 (Tool Calling)

모델이 외부 함수를 호출할 수 있도록 설정:

In [ ]:
from langchain_openai import ChatOpenAI

def get_weather(location: str) -> str:
    """특정 위치의 날씨를 가져옵니다."""
    return f"{location}는 화창합니다!"

model = ChatOpenAI(model="gpt-4.1-nano")

# 도구 바인딩
model_with_tools = model.bind_tools([get_weather])

# 호출
response = model_with_tools.invoke("샌프란시스코 날씨는?")
print(response.tool_calls)

### [연습문제] 모델 활용

**문제**: 다음 요구사항을 만족하는 코드를 작성하세요.

1. `gpt-4.1-mini` 모델을 초기화하세요.
2. "인공지능의 역사를 간단히 설명해주세요."라는 질문을 스트리밍 방식으로 처리하세요.
3. 전체 응답을 하나의 `AIMessage` 객체로 결합하세요.
4. 토큰 사용량을 출력하세요.

In [ ]:
# 여기에 코드를 작성하세요.


<details>
<summary>✅ 예시 정답</summary>

```python
from langchain_openai import ChatOpenAI

# 1. 모델 초기화
model = ChatOpenAI(model="gpt-4.1-mini")

# 2. 스트리밍 방식으로 처리
print("🤖 AI 응답:\n")
full_message = None
for chunk in model.stream("인공지능의 역사를 간단히 설명해주세요."):
    print(chunk.text, end="", flush=True)
    # 3. AIMessage 객체로 결합
    full_message = chunk if full_message is None else full_message + chunk

print("\n\n" + "="*50)

# 4. 토큰 사용량 출력
if full_message and full_message.usage_metadata:
    usage = full_message.usage_metadata
    print(f"📊 토큰 사용량:")
    print(f"  입력 토큰: {usage.get('input_tokens', 0)}")
    print(f"  출력 토큰: {usage.get('output_tokens', 0)}")
    print(f"  총 토큰: {usage.get('total_tokens', 0)}")
else:
    print("토큰 사용량 정보를 사용할 수 없습니다.")
```

</details>

---

### 3.2 메시지 (Messages)

LangChain은 다양한 메시지 타입을 제공하여 대화를 구조화합니다.


#### 3.2.1 HumanMessage (사용자 메시지)

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano")

# HumanMessage 생성
human_message = HumanMessage(content="Glory를 한국어로 번역해주세요.")

# 모델 호출
response = model.invoke([human_message])
print(f"번역: {response.content}")

#### 3.2.2 AIMessage (AI 응답)

In [ ]:
# AI 응답은 자동으로 AIMessage로 반환됨
response = model.invoke("안녕하세요!")

print(f"타입: {type(response)}")  # AIMessage
print(f"내용: {response.content}")
print(f"토큰 사용량: {response.usage_metadata}")

#### 3.2.3 SystemMessage (시스템 메시지)

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

# 시스템 메시지로 역할 정의
system_msg = SystemMessage(
    content="당신은 영어를 한국어로 번역하는 전문 번역가입니다."
)
human_msg = HumanMessage(content="Artificial Intelligence")

# 메시지 리스트로 호출
messages = [system_msg, human_msg]
response = model.invoke(messages)
print(f"번역: {response.content}")

#### 3.2.4 ToolMessage (도구 실행 결과)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.tools import tool

@tool
def get_weather(location: str) -> str:
    """특정 위치의 날씨를 가져옵니다."""
    return f"{location}는 화창합니다!"

model = ChatOpenAI(model="gpt-4.1-nano")

# 도구 바인딩
model_with_tools = model.bind_tools([get_weather])

# 호출
ai_message = model_with_tools.invoke("샌프란시스코 날씨는?")

# 도구 호출 결과 출력
print(ai_message.tool_calls)

In [ ]:
# 도구 실행 결과 출력

tool_message = get_weather.invoke(ai_message.tool_calls[0])

tool_message

In [ ]:
# 대화 계속
messages = [
    HumanMessage("샌프란시스코 날씨는?"),
    ai_message,
    tool_message
]
final_response = model.invoke(messages)
print(final_response.content)

### [연습문제] 메시지 활용

**문제**: 다음 시나리오를 구현하세요. (`gpt-4.1-nano` 모델 사용)

1. 시스템 메시지: "당신은 전문 수학 선생님입니다. 학생들에게 친절하게 설명해주세요."
2. 사용자가 "피타고라스 정리가 뭐예요?"라고 질문
3. AI가 응답
4. 사용자가 "예시를 들어주세요"라고 추가 질문
5. 전체 대화 내역을 출력

In [ ]:
# 여기에 코드를 작성하세요.


<details>
<summary>✅ 예시 정답</summary>

```python
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano")

# 대화 시작
messages = [
    SystemMessage(content="당신은 전문 수학 선생님입니다. 학생들에게 친절하게 설명해주세요."),
    HumanMessage(content="피타고라스 정리가 뭐예요?")
]

# 첫 번째 응답
response1 = model.invoke(messages)
print(f"🤖 선생님: {response1.content}\n")

# 대화 내역에 추가
messages.append(response1)
messages.append(HumanMessage(content="예시를 들어주세요"))

# 두 번째 응답
response2 = model.invoke(messages)
print(f"🤖 선생님: {response2.content}\n")

# 전체 대화 내역 출력
print("="*60)
print("📝 전체 대화 내역:")
print("="*60)
for i, msg in enumerate(messages, 1):
    role = msg.__class__.__name__
    print(f"\n[{i}] {role}:")
    print(f"    {msg.content[:100]}..." if len(msg.content) > 100 else f"    {msg.content}")
```

</details>

---

### 3.3 프롬프트 템플릿 (Prompt Templates)

프롬프트 템플릿은 동적 프롬프트를 생성하는 도구입니다.

#### 3.3.1 기본 프롬프트 템플릿

In [ ]:
from langchain_core.prompts import PromptTemplate

# 템플릿 생성
template = PromptTemplate.from_template("{topic}에 대한 이야기를 해줘")

# 템플릿 사용
prompt = template.invoke({"topic": "고양이"})
print(prompt)

#### 3.3.2 채팅 프롬프트 템플릿

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 여러 메시지를 포함하는 템플릿
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 도움이 되는 비서입니다"),
    ("user", "{subject}에 대해 설명해주세요")
])

# 사용
prompt = template.invoke({"subject": "양자역학"})
print(prompt.messages)

#### 3.3.3 메시지 플레이스홀더

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

# 대화 기록을 삽입할 수 있는 템플릿
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 도움이 되는 비서입니다"),
    MessagesPlaceholder("chat_history"),
    ("user", "{input}")
])

# 사용
prompt = template.invoke({
    "chat_history": [
        HumanMessage(content="안녕하세요!"),
    ],
    "input": "오늘 날씨 어때요?"
})

print(prompt.messages)

#### 3.3.4 체인 구성

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 컴포넌트 정의
prompt = PromptTemplate.from_template("도시 {city}의 특징을 알려주세요")
model = ChatOpenAI(model='gpt-4.1-nano')
parser = StrOutputParser()

# 체인 구성 (LCEL: LangChain Expression Language)
chain = prompt | model | parser

# 실행
result = chain.invoke({"city": "파리"})
print(result)

### [연습문제] 프롬프트 템플릿 실습

**문제**: 다음 요구사항을 만족하는 레시피 추천 시스템을 만드세요.

1. `ChatPromptTemplate`를 사용하여 다음 구조의 템플릿 생성:
   - 시스템 메시지: "당신은 전문 요리사입니다. 재료를 기반으로 레시피를 추천합니다."
   - 사용자 입력: "{ingredients}로 만들 수 있는 요리를 추천해주세요."
2. 프롬프트 → 모델 → 파서로 이어지는 체인 구성
3. "토마토, 계란, 치즈"를 입력으로 실행

In [ ]:
# 여기에 코드를 작성하세요.


<details>
<summary>✅ 예시 정답</summary>

```python
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 1. 프롬프트 템플릿 생성
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 전문 요리사입니다. 재료를 기반으로 레시피를 추천하고, 간단한 조리 방법도 알려줍니다."),
    ("user", "{ingredients}로 만들 수 있는 요리를 추천해주세요. 레시피와 조리 방법도 함께 알려주세요.")
])

# 모델과 파서
model = ChatOpenAI(model='gpt-4.1-nano', temperature=0.7)
parser = StrOutputParser()

# 2. 체인 구성
recipe_chain = template | model | parser

# 3. 실행
ingredients = "토마토, 계란, 치즈"
print(f"🥘 재료: {ingredients}\n")
print("="*60)

result = recipe_chain.invoke({"ingredients": ingredients})
print(result)
```

</details>

---

### 3.4 출력 파서 (Output Parsers)

모델의 텍스트 출력을 구조화된 데이터로 변환합니다.

#### 3.4.1 문자열 출력 파서

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# 체인에 통합
chain = prompt | model | parser
result = chain.invoke({"city": "도쿄"})

# 결과는 문자열
print(type(result))  # <class 'str'>
print(result)

#### 3.4.2 구조화된 출력

- **Pydantic을 사용한 구조화된 출력:**

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class CityInfo(BaseModel):
    """도시 정보 스키마"""
    name: str = Field(description="도시 이름")
    population: int = Field(description="인구 수")
    famous_landmarks: List[str] = Field(description="유명 랜드마크")

# 모델에 구조 바인딩
model_with_structure = model.with_structured_output(CityInfo)

# 호출
result = model_with_structure.invoke("파리에 대해 알려줘")
print(type(result))  # <class 'CityInfo'>
print(f"도시: {result.name}")
print(f"인구: {result.population:,}명")
print(f"랜드마크: {', '.join(result.famous_landmarks)}")

### [연습문제] 출력 파서 실습

**문제**: 영화 정보를 추출하는 시스템을 만드세요.

1. Pydantic `BaseModel`로 다음 필드를 가진 `MovieInfo` 스키마 정의:
   - `title`: 영화 제목 (str)
   - `genre`: 장르 (str)
   - `release_year`: 개봉 연도 (int)
   - `rating`: 평점 (float, 0-10 사이)
   - `summary`: 간단한 줄거리 (str)

2. 모델에 구조를 바인딩하고 "인셉션 영화에 대해 알려줘"로 테스트

In [ ]:
# 여기에 코드를 작성하세요.


<details>
<summary>✅ 예시 정답</summary>

```python
from pydantic import BaseModel, Field
from typing import Literal
from langchain_openai import ChatOpenAI

# 1. 스키마 정의
class MovieInfo(BaseModel):
    """영화 정보 스키마"""
    title: str = Field(description="영화 제목")
    genre: str = Field(description="영화 장르 (예: SF, 액션, 드라마)")
    release_year: int = Field(description="개봉 연도")
    rating: float = Field(description="평점 (0-10 사이)", ge=0, le=10)
    summary: str = Field(description="간단한 줄거리 (2-3문장)")

# 2. 모델 초기화 및 구조 바인딩
model = ChatOpenAI(model='gpt-4.1-nano')
structured_model = model.with_structured_output(MovieInfo)

# 3. 테스트
result = structured_model.invoke("인셉션 영화에 대해 알려줘")

print("🎬 영화 정보:")
print("="*60)
print(f"제목: {result.title}")
print(f"장르: {result.genre}")
print(f"개봉연도: {result.release_year}")
print(f"평점: {result.rating}/10")
print(f"\n줄거리:\n{result.summary}")
```

</details>

---

### 3.5 도구 (Tools)

도구는 에이전트가 외부 기능을 호출할 수 있게 해줍니다.

#### 3.5.1 기본 도구 정의

In [ ]:
from langchain.tools import tool

@tool
def add(a: float, b: float) -> float:
    """두 숫자를 더합니다."""
    return a + b

@tool
def multiply(a: float, b: float) -> float:
    """두 숫자를 곱합니다."""
    return a * b

# 도구 정보 확인
print(f"도구 이름: {add.name}")
print(f"도구 설명: {add.description}")
print(f"도구 인자: {add.args}")

#### 3.5.2 도구와 에이전트 통합

In [ ]:
from langchain.agents import create_agent

# 도구 목록
tools = [add, multiply]

# 에이전트 생성
agent = create_agent(
    model="gpt-4.1-nano",
    tools=tools,
    system_prompt="당신은 수학 계산을 도와주는 어시스턴트입니다."
)

# 실행
result = agent.invoke({
    "messages": [{"role": "user", "content": "15 곱하기 7은 얼마인가요?"}]
})

print(result['messages'][-1].content)

### [연습문제] 도구 활용

**문제**: 간단한 계산기 에이전트를 만드세요.

1. 다음 도구들을 정의하세요:
   - `add`: 두 수를 더하기
   - `subtract`: 두 수를 빼기
   - `multiply`: 두 수를 곱하기
   - `divide`: 두 수를 나누기 (0으로 나누기 예외 처리 포함)

2. 이 도구들을 사용하는 에이전트를 생성하세요.

3. 다음 질문들로 테스트하세요:
   - "25 더하기 17은?"
   - "100에서 37을 빼면?"
   - "12 곱하기 8은?"
   - "144를 12로 나누면?"

In [ ]:
# 여기에 코드를 작성하세요.


<details>
<summary>✅ 예시 정답</summary>

```python
from langchain.tools import tool
from langchain.agents import create_agent

# 1. 도구 정의
@tool
def add(a: float, b: float) -> float:
    """두 숫자를 더합니다."""
    return a + b

@tool
def subtract(a: float, b: float) -> float:
    """첫 번째 숫자에서 두 번째 숫자를 뺍니다."""
    return a - b

@tool
def multiply(a: float, b: float) -> float:
    """두 숫자를 곱합니다."""
    return a * b

@tool
def divide(a: float, b: float) -> str:
    """첫 번째 숫자를 두 번째 숫자로 나눕니다."""
    if b == 0:
        return "오류: 0으로 나눌 수 없습니다."
    result = a / b
    return f"{result:.2f}"

# 2. 에이전트 생성
tools = [add, subtract, multiply, divide]

agent = create_agent(
    model="gpt-4.1-nano",
    tools=tools,
    system_prompt="당신은 친절한 수학 계산기 어시스턴트입니다. 사용자의 계산 요청을 정확히 처리해주세요."
)

# 3. 테스트
test_questions = [
    "25 더하기 17은?",
    "100에서 37을 빼면?",
    "12 곱하기 8은?",
    "144를 12로 나누면?",
    "복잡한 계산: (25 + 15) 곱하기 2는?"
]

print("🧮 계산기 에이전트 테스트\n" + "="*60)

for question in test_questions:
    print(f"\n❓ 질문: {question}")
    result = agent.invoke({
        "messages": [{"role": "user", "content": question}]
    })
    print(f"✅ 답변: {result['messages'][-1].content}")
```

</details>

---

## 4. 에이전트 구축

- **에이전트(Agent)** 는 LLM을 사용하여 다음을 수행하는 시스템

    1. 목표를 이해하고 계획 수립
    2. 필요한 도구를 선택하고 실행
    3. 결과를 평가하고 다음 단계 결정
    4. 목표 달성까지 반복


---

### 4.1 create_agent로 에이전트 생성

#### 4.1.1 기본 에이전트

In [ ]:
from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """주어진 도시의 날씨를 가져옵니다."""
    return f"{city}는 화창합니다!"

agent = create_agent(
    model="gpt-4.1-nano",
    tools=[get_weather],
    system_prompt="당신은 도움이 되는 어시스턴트입니다."
)

# 실행
result = agent.invoke({
    "messages": [{"role": "user", "content": "샌프란시스코 날씨는?"}]
})

for msg in result['messages']:
    msg.pretty_print()

#### 4.1.2 다중 도구 에이전트

In [ ]:
from langchain.tools import tool
from langchain.agents import create_agent

@tool
def search_web(query: str) -> str:
    """웹에서 정보를 검색합니다."""
    return f"'{query}'에 대한 검색 결과: [가상의 검색 결과]"

@tool
def calculate(expression: str) -> str:
    """수학 표현식을 계산합니다."""
    try:
        result = eval(expression)
        return f"계산 결과: {result}"
    except:
        return "계산 오류"

@tool
def translate(text: str, target_lang: str = "한국어") -> str:
    """텍스트를 번역합니다."""
    return f"'{text}'를 {target_lang}로 번역: [번역 결과]"

# 다중 도구 에이전트
agent = create_agent(
    model="gpt-4.1-mini",
    tools=[search_web, calculate, translate],
    system_prompt="""당신은 다재다능한 어시스턴트입니다.
    
    사용 가능한 도구:
    - search_web: 정보 검색
    - calculate: 계산
    - translate: 번역
    
    사용자의 요청에 따라 적절한 도구를 선택하여 사용하세요."""
)

# 테스트
result = agent.invoke({
    "messages": [{"role": "user", "content": "25 * 18을 계산하고, 결과를 일본어로 번역해줘"}]
})


for msg in result["messages"]:
    msg.pretty_print()

---

### 4.2 구조화된 출력 (Structured Output)

에이전트가 특정 형식으로 응답하도록 설정할 수 있습니다.

#### 4.2.1 ToolStrategy

In [ ]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

class ContactInfo(BaseModel):
    """연락처 정보"""
    name: str = Field(description="이름")
    email: str = Field(description="이메일")
    phone: str = Field(description="전화번호")

agent = create_agent(
    model="gpt-4.1-mini",
    tools=[search_web],
    response_format=ToolStrategy(ContactInfo)
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "연락처 추출: 홍길동, hong@example.com, 010-1234-5678"}]
})

print(result["structured_response"])


#### 4.2.2 ProviderStrategy

제공자의 네이티브 구조화 출력을 사용:

In [ ]:
from langchain.agents.structured_output import ProviderStrategy

agent = create_agent(
    model="gpt-4.1-mini",  # 네이티브 구조화 출력 지원 모델
    tools=[],
    response_format=ProviderStrategy(ContactInfo)
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "연락처 추출: 홍길동, hong@example.com, 010-1234-5678"}]
})

print(result["structured_response"])

---

### 4.3 메모리와 컨텍스트

#### 4.3.1 대화 메모리

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# 메모리 저장소
memory = MemorySaver()

agent = create_agent(
    model="gpt-4.1-nano",
    tools=[],
    checkpointer=memory
)

# 첫 번째 대화
config = {"configurable": {"thread_id": "user-123"}}
agent.invoke(
    {"messages": [{"role": "user", "content": "내 이름은 김철수야"}]},
    config=config
)

In [ ]:
# 두 번째 대화 (같은 thread_id)
result = agent.invoke(
    {"messages": [{"role": "user", "content": "내 이름이 뭐였지?"}]},
    config=config
)

for msg in result["messages"]:
    msg.pretty_print()

### [연습문제] 에이전트 구축

**문제**: 다음 기능을 가진 "개인 비서 에이전트"를 만드세요.

1. **도구들**:
   - `get_current_time()`: 현재 시간 반환
   - `set_reminder(task: str, time: str)`: 리마인더 설정
   - `search_contact(name: str)`: 연락처 검색
   - `send_email(to: str, subject: str, body: str)`: 이메일 발송

2. **구조화된 출력**: 
   ```python
   class AssistantResponse(BaseModel):
       action_taken: str
       result: str
       next_suggestion: str
   ```

3. **테스트 시나리오**:
   - "현재 시간 알려줘"
   - "내일 오전 10시에 회의 리마인더 설정해줘"
   - "홍길동 연락처 찾아줘"

In [ ]:
# 여기에 코드를 작성하세요.
from datetime import datetime
from pydantic import BaseModel, Field
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

# 1. 도구 정의
@tool
def get_current_time() -> str:
    """현재 시간을 반환합니다."""
    now = datetime.now()
    return now.strftime("%Y년 %m월 %d일 %H시 %M분")

@tool
def set_reminder(task: str, time: str) -> str:
    """리마인더를 설정합니다."""
    return f"✅ 리마인더 설정됨: '{task}' - {time}"

@tool
def search_contact(name: str) -> str:
    """연락처를 검색합니다."""
    # 가상의 연락처 데이터베이스
    contacts = {
        "홍길동": {"phone": "010-1234-5678", "email": "hong@example.com"},
        "김철수": {"phone": "010-9876-5432", "email": "kim@example.com"},
    }
    
    if name in contacts:
        contact = contacts[name]
        return f"📞 {name}: {contact['phone']}, {contact['email']}"
    return f"❌ '{name}' 연락처를 찾을 수 없습니다."

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """이메일을 발송합니다."""
    return f"📧 이메일 발송됨:\n  수신: {to}\n  제목: {subject}\n  내용: {body[:50]}..."

# 2. 구조화된 출력 스키마
class AssistantResponse(BaseModel):
    """비서 응답 형식"""
    action_taken: str = Field(description="수행한 작업")
    result: str = Field(description="작업 결과")
    next_suggestion: str = Field(description="다음 제안사항")

# 3. 에이전트 생성
tools = [get_current_time, set_reminder, search_contact, send_email]

agent = create_agent(
    model="gpt-4.1-mini",
    tools=tools,
    system_prompt="""당신은 친절하고 능력있는 개인 비서입니다.
    
    사용자의 요청을 정확히 이해하고, 적절한 도구를 사용하여 업무를 처리하세요.
    항상 정중하고 효율적으로 응대하며, 추가로 도움이 될 만한 제안을 해주세요.""",
    response_format=ToolStrategy(AssistantResponse)
)

# 4. 테스트
test_scenarios = [
    "현재 시간 알려줘",
    "내일 오전 10시에 프로젝트 회의 리마인더 설정해줘",
    "홍길동 연락처 찾아줘",
    "김철수에게 '안부 인사'라는 제목으로 '잘 지내시죠?'라는 내용의 이메일 보내줘"
]

print("🤖 개인 비서 에이전트 테스트")
print("="*70 + "\n")

for scenario in test_scenarios:
    print(f"📝 요청: {scenario}")
    result = agent.invoke({
        "messages": [{"role": "user", "content": scenario}]
    })
    
    response = result["structured_response"]
    print(f"  작업: {response.action_taken}")
    print(f"  결과: {response.result}")
    print(f"  제안: {response.next_suggestion}")
    print("-"*70 + "\n")

<details>
<summary>✅ 예시 정답</summary>

```python
from datetime import datetime
from pydantic import BaseModel, Field
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy

# 1. 도구 정의
@tool
def get_current_time() -> str:
    """현재 시간을 반환합니다."""
    now = datetime.now()
    return now.strftime("%Y년 %m월 %d일 %H시 %M분")

@tool
def set_reminder(task: str, time: str) -> str:
    """리마인더를 설정합니다."""
    return f"✅ 리마인더 설정됨: '{task}' - {time}"

@tool
def search_contact(name: str) -> str:
    """연락처를 검색합니다."""
    # 가상의 연락처 데이터베이스
    contacts = {
        "홍길동": {"phone": "010-1234-5678", "email": "hong@example.com"},
        "김철수": {"phone": "010-9876-5432", "email": "kim@example.com"},
    }
    
    if name in contacts:
        contact = contacts[name]
        return f"📞 {name}: {contact['phone']}, {contact['email']}"
    return f"❌ '{name}' 연락처를 찾을 수 없습니다."

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """이메일을 발송합니다."""
    return f"📧 이메일 발송됨:\n  수신: {to}\n  제목: {subject}\n  내용: {body[:50]}..."

# 2. 구조화된 출력 스키마
class AssistantResponse(BaseModel):
    """비서 응답 형식"""
    action_taken: str = Field(description="수행한 작업")
    result: str = Field(description="작업 결과")
    next_suggestion: str = Field(description="다음 제안사항")

# 3. 에이전트 생성
tools = [get_current_time, set_reminder, search_contact, send_email]

agent = create_agent(
    model="gpt-4.1-mini",
    tools=tools,
    system_prompt="""당신은 친절하고 능력있는 개인 비서입니다.
    
    사용자의 요청을 정확히 이해하고, 적절한 도구를 사용하여 업무를 처리하세요.
    항상 정중하고 효율적으로 응대하며, 추가로 도움이 될 만한 제안을 해주세요.""",
    response_format=ToolStrategy(AssistantResponse)
)

# 4. 테스트
test_scenarios = [
    "현재 시간 알려줘",
    "내일 오전 10시에 프로젝트 회의 리마인더 설정해줘",
    "홍길동 연락처 찾아줘",
    "김철수에게 '안부 인사'라는 제목으로 '잘 지내시죠?'라는 내용의 이메일 보내줘"
]

print("🤖 개인 비서 에이전트 테스트")
print("="*70 + "\n")

for scenario in test_scenarios:
    print(f"📝 요청: {scenario}")
    result = agent.invoke({
        "messages": [{"role": "user", "content": scenario}]
    })
    
    response = result["structured_response"]
    print(f"  작업: {response.action_taken}")
    print(f"  결과: {response.result}")
    print(f"  제안: {response.next_suggestion}")
    print("-"*70 + "\n")
```

</details>